# BikewaySim

Use this jupyter notebook to run BikewaySim. BikewaySim finds the shortest path between any origin and destination using Dijkstra's algorithm. 

BikewaySim requires the following before running:

- A nodes shapefile/geojson with node ids.
- A links shapefile/geojson with refrences ids (i.e. column that specifies the starting and ending node id)
- A csv with origin and destination pairs

Notes:
- Run the convert_to_bikewaysim_network script before running bikewaysim to make sure your network is compatible
- The desired link cost should be specifed in the "distance" column of the links. If you want to specify your own link costs using link attributes, then modify the "distance" column before using the network in BikewaySim

Other Notes:

Use Python 3.7+


The following third-party packages are used in the BikewaySim module:
```python
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point

import networkx as nx

import matplotlib.pyplot as plt
```

# Preparation

Import modules/packages required for this notebook.

In [1]:
import os
from pathlib import Path

import numpy as np
import pandas as pd
import geopandas as gpdui

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/%autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2
pd.options.display.max_columns = None  # display all columns

In [2]:
# import TransportSim module
from core_script.getPaths_transportSim import *

### Setting the directory

In [3]:
# need to set this environmental path for network data and query data at separate locations
user_directory = os.fspath(Path.home()) #get home directory and convert to path string

# set path variable for BikewaySim
os.environ['PROJ_LIB'] = user_directory + "/Documents/GitHub/BikewaySimDev"

# network shapefile data path directory
#os.environ['bws_NETWORK'] = user_directory + r"/Documents/BikewaySimData/processed_shapefiles/prepared_network/dist"
os.environ['bws_NETWORK'] = user_directory + r"/Documents/BikewaySimData/processed_shapefiles/prepared_network/per_dist"

# OD Pairs directory and location of output files
os.environ['bws_TRIPS'] = user_directory + "/Documents/BikewaySimData/bikewaysim_outputs"

# create BikewaySim object for running 'bikewaysim' option
bws = BikewaySim(os.environ['PROJ_LIB'], os.environ['bws_NETWORK'], os.environ['bws_TRIPS'])

Initialize BikewaySim object to process sidewalk graph and pre-process trip queries!
Set/Update BikewaySim dict_settings!


### Prepare networks for BikewaySim

In [4]:
# prepare networks
df_links, dict_bike = bws.prepare_network(grid_size=10000)
# print default settings

** Initialize BikewaySim network **
It takes 33.404 seconds to run the function 'prepare_network()'!


In [5]:
# strategy: 1. given origin time find earliest arrival
#           2. given expected arrival time find latest departure time
dict_settings = {'walk_speed': 2.0,  # people's walking speed 2 mph
                 'grid_size': 10000.0,  # for searching nearby links by grouping links to grids with width 10000 ft. for efficiency in searching
                 'ntp_dist_thresh': 5280.0,  # node to point (maximum distance access to network from origin/destination); (walking) distance threshold
                 'network': {'bike': dict_bike},  # dump in networks and modes
                 # strategy determines network link's direction.
                 # Strategy 1: Find earliest arrival given query time as departure time
                 # Strategy 2: Find latest departure time given query time as arrival time
             
                 'strategy': {'bike': 1},  # 1. find earliest arrival 2. find latest departure
                 'query_time': [8],  # departure time or arrival time of a trip, depends on the strategy
                 
                 'walk_thresh': {'bike': 0.5},  # walking threshold is 0.3 miles
                 'num_options': {'bike': 1},  # if set to 2, return 2-shortest paths
                 'plot_all': False,  # if True, plot results and save plots for all routes found #this is broken now
                 'one_by_one': False  # set time and strategy one by one
                 }
# load dict_settings to the sws object
bws.dict_settings = dict_settings

Set/Update BikewaySim dict_settings!


In [6]:
df_points = bws.prepare_trips(option='bike')
display(df_points.head())

** load trip data & prepare sample **
It takes 20.298 seconds to run the function 'prepare_trips()'!


,trip_id,ori_id,dest_id,ori_lat,ori_lon,dest_lat,dest_lon,geometry,ox,oy,ox_sq,oy_sq,o_node,o_t,o_d,dx,dy,dx_sq,dy_sq,d_node,d_t,d_d
0,538_376,538,376,33.789796,-84.362088,33.793559,-84.403807,POINT (2224539.768 1380026.598),2.237211e+06,1.378631e+06,224.0,138.0,3069322975,0.011662,0.023325,2.224540e+06,1.380027e+06,222.0,138.0,30283193156,0.081293,0.162586
1,538_478,538,478,33.789796,-84.362088,33.777577,-84.409624,POINT (2222758.914 1374215.011),2.237211e+06,1.378631e+06,224.0,138.0,3069322975,0.011662,0.023325,2.222759e+06,1.374215e+06,222.0,137.0,305407044385,0.056166,0.112331
2,538_479,538,479,33.789796,-84.362088,33.778119,-84.407853,POINT (2223297.590 1374410.970),2.237211e+06,1.378631e+06,224.0,138.0,3069322975,0.011662,0.023325,2.223298e+06,1.374411e+06,222.0,137.0,301117527931,0.011749,0.023498
3,538_480,538,480,33.789796,-84.362088,33.777463,-84.404200,POINT (2224406.827 1374169.709),2.237211e+06,1.378631e+06,224.0,138.0,3069322975,0.011662,0.023325,2.224407e+06,1.374170e+06,222.0,137.0,3069175002,0.006422,0.012844
4,538_481,538,481,33.789796,-84.362088,33.774730,-84.405087,POINT (2224135.207 1373176.145),2.237211e+06,1.378631e+06,224.0,138.0,3069322975,0.011662,0.023325,2.224135e+06,1.373176e+06,222.0,137.0,30546641348,0.006141,0.012283


In [7]:
# run all queries
bws.run_all()

**for each trip, find k-shortest paths**
PATH #: 0
['3069322975', '3069322962', '3069322952', '3069322946', '3069322939', '306003256470', '302508783164', '302351305340', '304122397843', '301926466428', '306003263553', '302727216050', '302727216057', '302727216051', '302727216047', '309511019511', '309511019512', '305839157341', '305839157350', '302349760557', '3069332025', '303376411414', '305350824287', '301503611146', '3069507413', '3069507410', '308481868553', '3069388157', '3069507392', '307848406331', '3069179451', '305996575300', '3069179460', '3069461868', '3069507339', '305998998736', '3069507309', '305998998748', '3069392950', '305998998750', '30761543364', '304165941638', '3069392941', '302641424453', '307851189575', '3069290382', '307851189576', '306694977826', '306694977825', '3069290388', '309536158678', '301278546121', '306005931712', '3069290394', '305999006952', '3069372832', '305999006949', '305999006961', '3069616413', '305999006955', '30283192513', '305999241048', '3

PATH #: 0
['3069322975', '3069322962', '3069322952', '3069322946', '3069322939', '306003256470', '302508783164', '302351305340', '304122397843', '301926466428', '306003263553', '302727216050', '308481455640', '308481455647', '302727216059', '303505311493', '304405559074', '3069332064', '305674504872', '305674504871', '306003256645', '306003256646', '302396740007', '302396740006', '306003256649', '306003256650', '306003256651', '306945805266', '302396740003', '301927417783', '306003256656', '306003256657', '309543424405', '306003256658', '306012323214', '3069614561', '306012323217', '306012326800', '3069539145', '3069644020', '3069515348', '3069350695', '3069531975', '306012326810', '307786122287', '308481415862', '306005931630', '3069531973', '30535741449', '3069531971', '306005931596', '305411918868', '305416479051', '304165942040', '3069515840', '304165942044', '305996559986', '3069531969', '3069531965', '3069372787', '30535078505', '303486632541', '303486632542', '301975681204', '30

PATH #: 0
['3069322975', '3069322962', '3069322952', '3069322946', '3069322939', '306003256470', '302508783164', '302351305340', '304122397843', '301926466428', '306003263553', '302727216050', '308481455640', '308481455647', '302727216059', '303505311493', '304405559074', '3069332064', '305674504872', '305674504871', '306003256645', '306003256646', '302396740007', '302396740006', '306003256649', '306003256650', '306003256651', '306945805266', '302396740003', '301927417783', '306003256656', '306003256657', '309543424405', '306003256658', '306012323214', '3069614561', '306012323217', '306012326800', '3069539145', '3069644020', '3069515348', '3069350695', '3069531975', '306012326810', '307786122287', '308481415862', '306005931630', '3069531973', '30535741449', '3069531971', '306005931596', '305411918868', '305416479051', '304165942040', '3069515840', '304165942044', '305996559986', '3069531969', '3069531965', '3069372787', '3069372259', '305996555227', '3069365771', '305996555229', '30599

PATH #: 0
['3069322975', '3069322962', '3069322952', '3069322946', '3069322939', '306003256470', '302508783164', '302351305340', '304122397843', '301926466428', '306003263553', '302727216050', '308481455640', '308481455647', '302727216059', '303505311493', '304405559074', '3069332064', '305674504872', '305674504871', '306003256645', '306003256646', '302396740007', '302396740006', '306003256649', '306003256650', '306003256651', '306945805266', '302396740003', '301927417783', '306003256656', '306003256657', '309543424405', '306003256658', '306012323214', '3069614561', '306012323217', '306012326800', '3069539145', '3069644020', '3069515348', '3069350695', '3069531975', '306012326810', '307786122287', '308481415862', '306005931630', '3069531973', '30535741449', '3069531971', '306005931596', '305411918868', '305416479051', '304165942040', '3069515840', '304165942044', '305996559986', '3069531969', '3069531965', '3069372787', '30535078505', '303486632541', '303486632542', '301975681204', '30

PATH #: 0
['3069322975', '3069322962', '3069322952', '3069322946', '3069322939', '306003256470', '302508783164', '302351305340', '304122397843', '301926466428', '306003263553', '302727216050', '308481455640', '308481455647', '302727216059', '303505311493', '304405559074', '3069332064', '305674504872', '305674504871', '306003256645', '306003256646', '302396740007', '302396740006', '306003256649', '306003256650', '306003256651', '306945805266', '302396740003', '301927417783', '306003256656', '306003256657', '309543424405', '306003256658', '306012323214', '3069614561', '306012323217', '306012326800', '3069539145', '3069644020', '3069515348', '3069350695', '3069531975', '306012326810', '307786122287', '308481415862', '306005931630', '3069531973', '30535741449', '3069531971', '306005931596', '305411918868', '305416479051', '304165942040', '3069515840', '304165942044', '305996559986', '3069531969', '3069531965', '3069372787', '30535078505', '303486632541', '303486632542', '301975681204', '30

...

PATH #: 0
['3069322975', '3069322962', '3069322952', '3069322946', '3069322939', '306003256470', '302508783164', '302351305340', '304122397843', '301926466428', '306003263553', '302727216050', '308481455640', '308481455647', '302727216059', '303505311493', '304405559074', '3069332064', '305674504872', '305674504871', '304879481811', '303376386336', '302256317495', '30969139261', '3069331813', '304310132552', '303158138037', '303158138038', '3069331815', '3069331817', '3069331820', '306945790460', '3069331827', '3069331831', '304879481743', '3069331836', '30969139217', '301928281303', '301928281253', '3069331850', '301975639623', '301975639659', '30344455889', '30568595966', '30344455899', '30344455920', '301080552724', '30344455931', '30344456091', '306016077265', '30344456113', '30344456158', '30926733449', '30344456220', '30926733460', '308202468354', '30926766945', '30344456236', '30344456241', '30344456219', '3069245200', '3069398081', '306014094963', '3069398078', '306014094959', 

PATH #: 0
['3069322975', '3069322962', '3069322952', '3069322946', '3069322939', '306003256470', '302508783164', '302351305340', '304122397843', '301926466428', '306003263553', '302727216050', '308481455640', '308481455647', '302727216059', '303505311493', '304405559074', '3069332064', '305674504872', '305674504871', '306003256645', '306003256646', '302396740007', '302396740006', '306003256649', '306003256650', '306003256651', '306945805266', '302396740003', '301927417783', '306003256656', '306003256657', '309543424405', '306003256658', '306012323214', '3069614561', '306012323217', '306012326800', '3069539145', '3069644020', '3069515348', '3069350695', '3069531975', '306012326814', '3069365786', '3069228780', '305416479038', '305416479027', '306012323191', '3069436067', '306012322971', '305416479031', '306016350512', '3069121376', '306016353288', '305410853279', '306016350518', '305410853278', '305416381270', '305410853277', '308147553458', '305416381272', '305410853274', '305424210067

PATH #: 0
['3069322975', '3069322962', '3069322952', '3069322946', '3069322939', '306003256470', '302508783164', '302351305340', '304122397843', '301926466428', '306003263553', '302727216050', '308481455640', '308481455647', '302727216059', '303505311493', '304405559074', '3069332064', '305674504872', '305674504871', '306003256645', '306003256646', '302396740007', '302396740006', '306003256649', '306003256650', '306003256651', '306945805266', '302396740003', '301927417783', '306003256656', '306003256657', '309543424405', '306003256658', '306012323214', '3069614561', '306012323217', '306012326800', '3069539145', '3069644020', '3069515348', '3069350695', '3069531975', '306012326810', '307786122287', '308481415862', '306005931630', '3069531973', '30535741449', '3069531971', '306005931596', '305411918868', '305416479051', '304165942040', '3069515840', '304165942044', '305996559986', '3069531969', '3069531965', '3069372787', '30535078505', '303486632541', '303486632542', '301975681204', '30

PATH #: 0
['3069322975', '3069261678', '3069223162', '3069323027', '3069323031', '306003237280', '3069323033', '3069323048', '306003237284', '306003237276', '3069528783', '306003237270', '3069229271', '306003230927', '302264674019', '3069229278', '3069229280', '306003237218', '3069229283', '306003237224', '306877187276', '3069367555', '306003230859', '3069545571', '30384327359', '3068225169', '301778962527', '301778962560', '3068192021', '306877549822', '305416080549', '305416113419', '305416113418', '305422168219', '305416113408', '305416113405', '305416113406', '306308020790', '305411997198', '305411997192', '305411997176', '305411997177', '305411997166']
PATH #: 0
['3069322975', '3069261678', '3069223162', '3069323027', '3069323031', '306003237280', '3069323033', '3069323048', '306003237284', '306003237276', '3069528783', '306003237270', '3069229271', '306003230927', '302264674019', '3069229278', '3069229280', '306003237218', '3069229283', '306003237224', '306877187276', '3069367555

PATH #: 0
['3069322975', '3069322962', '3069322952', '3069322946', '3069322939', '306003256470', '302508783164', '302351305340', '304122397843', '301926466428', '306003263553', '302727216050', '308481455640', '308481455647', '302727216059', '303505311493', '304405559074', '3069332064', '305674504872', '305674504871', '304879481811', '303376386336', '302256317495', '30969139261', '3069331813', '304310132552', '303158138037', '303158138038', '3069331815', '3069331817', '3069331820', '306945790460', '3069331827', '3069331831', '304879481743', '3069331836', '30969139217', '301928281303', '301928281253', '3069331850', '301975639623', '301975639659', '30344455889', '30267605628', '30267605626', '30925984966', '301684496815', '306016047139', '306016047156', '3069435707', '3069435710', '302279063071', '3069282106', '3069282108', '305422197634', '301088817925', '3069253109', '3069230439', '3069230556', '306908434299', '307561437462', '3069253107', '3069462724', '3069222583', '3069133263', '3025

# Results

The trip inputs & results are stored in ' ***/trips_bws/results*** '

### Columns defintion of the output results
Every row correpsonds to a link<br>
**A**: origin node id<br>
**B**: destination node id<br>
**dist**: distances<br>
**mode**: mode of transportation (e.g., walk, bike, etc.)<br>
**strategy**: chosen strategy<br>
**route_num**: the k number of k-shortest routes return for each link<br>
**sequence**: the trip segment number in a sequence of trips<br>
**time**: time it takes to travel through the link<br>
**timeStamp**: accumulative time of the current trip (in hours)<br>
**trip_id**: trip id (from the trip query)<br>
**route**: if there is a crosswalk, which crosswalk it uses<br>

### For graph outputs

***BikeSim/build_graph/results_routes***

naming rule:

{trip_id}_{(k-1)th route}.PNG

e.g.,

{4001880_1_1}_{0}.PNG